# 1 - Libraries and past data download

In [6]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [7]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=59)).strftime('%Y-%m-%d')

In [8]:
dataF = yf.download("EURUSD=X", start=start_date, end=today, interval='15m')

[*********************100%***********************]  1 of 1 completed


In [9]:
dataF.tail(5)
# dataF.Open.iloc

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-09-20 21:15:00,1.116196,1.116196,1.115947,1.116071,1.116071,0
2024-09-20 21:30:00,1.116071,1.116570,1.115947,1.116570,1.116570,0
2024-09-20 21:45:00,1.116570,1.116695,1.116321,1.116695,1.116695,0
2024-09-20 22:00:00,1.116570,1.116570,1.116570,1.116570,1.116570,0
2024-09-20 22:15:00,1.116570,1.116570,1.116570,1.116570,1.116570,0


# 2 - Signal Generator function
Signals Buying, Selling or No Action

In [10]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if(open > close and
       previous_open < previous_close and
       close < previous_open and
       open >= previous_close):
        return 'buy'
    
    # Bullish Pattern
    elif(open < close and
         previous_open > previous_close and
         close > previous_open and
         open <= previous_close):
        return 'sell'
    
    # No clear pattern
    else:
        return 'none'  


Test the function:

In [11]:
signal = []
signal.append('none')
for i in range(1,dataF.shape[0]):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))

dataF['signal'] = signal

In [12]:
dataF.signal.value_counts()

signal
none    3585
sell     153
buy      145
Name: count, dtype: int64

In [13]:
dataF.signal.value_counts(normalize=True)*100

signal
none    92.325522
sell     3.940252
buy      3.734226
Name: proportion, dtype: float64

# 3 - Connect to the market and execute trades

In [1]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [2]:
# export JUPYTER_PATH=$JUPYTER_PATH:/home/luiz/Documents/config_tradingbot/
import sys
sys.path.append('/home/luiz/Documents/config_tradingbot/')

In [3]:
from config import access_token, accountID

In [4]:
# from config import access_token, accountID
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)

True
True
True


In [5]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [17]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

NameError: name 'access_token' is not defined